In [16]:
import os 
import pathlib
import pickle as pkl
import cloudpickle 
#import pyarrow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import itertools
import re
#import sklearn
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, RocCurveDisplay, average_precision_score, precision_recall_curve, PrecisionRecallDisplay
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer, StandardScaler, RobustScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
pd.set_option('display.max_columns', None)
from bill_classification import *

In [ ]:
X_train, X_test, y_train, y_test, y_classes, bill_ls = split_features()

In [ ]:
MLP_pipeline_fit(X_train, y_train, "../data/models/MLP_fit.pkl")

Fitting 5 folds for each of 27 candidates, totalling 135 fits


/work/DPF_exam/env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/work/DPF_exam/env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/work/DPF_exam/env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
#transformers
categorical_transformer=Pipeline(
    steps=[('encoder', OneHotEncoder(sparse_output=False))])
multi_categorical_transformer=Pipeline(
    steps=[('encoder', CountVectorizer(analyzer=lambda lst: lst))])
binary_transformer=Pipeline(
    steps=[('encoder', OneHotEncoder(sparse_output=False, drop="if_binary"))])
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())])
count_transformer = Pipeline(
    steps=[("scaler", RobustScaler())])

In [6]:
ct = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, ["state"]),
        ("multi_cat", multi_categorical_transformer, "ncsl_metatopics"),
        ("bin", binary_transformer, ["cha", "lobbied"]),
        ("num", numeric_transformer, make_column_selector(dtype_include="float64")), 
        ("count", count_transformer, make_column_selector(dtype_include="int32"))
    ],
    remainder='passthrough', 
    force_int_remainder_cols = False, sparse_threshold=0)